# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from vaastav.understat import *
from vaastav.getters import *
from src.match_names import *
from config import *

In [3]:
SEASONS = ["2021", "2022", "2023"]

# Getting data

In [4]:
fixtures = get_fixtures_data()

In [5]:
df_fixtures = pd.DataFrame(fixtures).dropna()

In [6]:
def team_id_to_name(current_teams, team_id):
    return current_teams[team_id - 1]

In [7]:
df_fixtures["Squad H"] = df_fixtures["team_h"].apply(lambda x: team_id_to_name(current_teams, x))
df_fixtures["Squad A"] = df_fixtures["team_a"].apply(lambda x: team_id_to_name(current_teams, x))
df_fixtures["date"] = df_fixtures["kickoff_time"].str[:10]

In [8]:
epl_seasons = []
for season in SEASONS:
    epl_season = get_epl_data(season)
    epl_season = epl_season[0]
    epl_seasons.append(epl_season)

In [9]:
# epl_data

# Fixture DF

In [10]:
df = pd.DataFrame(columns=["Team", "xG", "xGA", "Date", "GW"])

for season in epl_seasons:
    for k in season.keys():
        title = team_understat_to_fpl(season[k]['title'])
        for h in season[k]['history']:
            xG = h["xG"]
            xGA = h["xGA"]
            date = h["date"][:10]

            new_row = {
                        "Team": title,
                        "xG": xG,
                        "xGA": xGA,
                        "Date": date
                      }
            df = df.append(new_row, ignore_index=True)

In [11]:
def assign_opponent(x):
    r1 = df_fixtures[ (df_fixtures["date"] == x["Date"]) & (df_fixtures["Squad H"] == x["Team"]) ]
    r2 = df_fixtures[ (df_fixtures["date"] == x["Date"]) & (df_fixtures["Squad A"] == x["Team"]) ]
    if r1.size > 0:
        return r1["Squad A"].item()
    if r2.size > 0:
        return r2["Squad H"].item()
    return None

In [12]:
for i, row in df.iterrows():
    df.loc[i, "Opponent"] = assign_opponent(row)

In [13]:
df["Team"].apply(lambda x: team_understat_to_fpl(x)).unique()

array(['Aston Villa', 'Everton', 'Southampton', 'Leicester City',
       'Crystal Palace', 'Norwich City', 'Chelsea', 'West Ham',
       'Tottenham', 'Arsenal', 'Newcastle Utd', 'Liverpool',
       'Manchester City', 'Manchester Utd', 'Watford', 'Burnley',
       'Brighton', 'Wolves', 'Brentford', 'Leeds United', 'Bournemouth',
       'Fulham', "Nott'ham Forest", 'Sheffield Utd', 'Luton Town'],
      dtype=object)

In [14]:
df["Team"] = df["Team"].apply(lambda x: team_understat_to_fpl(x))

In [15]:
team_understat_to_fpl('Sheffield United')

'Sheffield Utd'

In [16]:
df["Team"].unique()

array(['Aston Villa', 'Everton', 'Southampton', 'Leicester City',
       'Crystal Palace', 'Norwich City', 'Chelsea', 'West Ham',
       'Tottenham', 'Arsenal', 'Newcastle Utd', 'Liverpool',
       'Manchester City', 'Manchester Utd', 'Watford', 'Burnley',
       'Brighton', 'Wolves', 'Brentford', 'Leeds United', 'Bournemouth',
       'Fulham', "Nott'ham Forest", 'Sheffield Utd', 'Luton Town'],
      dtype=object)

In [17]:
df = df.drop(columns=["GW", "Opponent"])

In [18]:
df

,Team,xG,xGA,Date
0,Aston Villa,1.137180,1.350360,2021-08-14
1,Aston Villa,1.184260,0.631630,2021-08-21
2,Aston Villa,0.431464,1.133120,2021-08-28
3,Aston Villa,1.173490,1.221530,2021-09-11
4,Aston Villa,0.777716,0.755035,2021-09-18
...,...,...,...,...
1575,Nott'ham Forest,1.371810,0.551038,2023-08-18
1576,Nott'ham Forest,0.960354,3.028030,2023-08-26
1577,Luton Town,1.885940,4.367480,2023-08-12
1578,Luton Town,0.465741,3.414190,2023-08-25


In [19]:
df["Team"].unique()

array(['Aston Villa', 'Everton', 'Southampton', 'Leicester City',
       'Crystal Palace', 'Norwich City', 'Chelsea', 'West Ham',
       'Tottenham', 'Arsenal', 'Newcastle Utd', 'Liverpool',
       'Manchester City', 'Manchester Utd', 'Watford', 'Burnley',
       'Brighton', 'Wolves', 'Brentford', 'Leeds United', 'Bournemouth',
       'Fulham', "Nott'ham Forest", 'Sheffield Utd', 'Luton Town'],
      dtype=object)

# Saving to file

In [20]:
df.to_csv("data/completed_fixtures.csv")